### BW39 - 객체를 제너릭하게 구성하려면 @classmethod를 통한 다형성을 활용하자

- 파이썬에서는 객체뿐만 아니라 클래스도 다형성을 지원한다.
<br/> → 계층을 이루는 여러 클래스가 자신에게 맞는 유일한 메서드 버전을 구현할 수 있다. (서로 다른 기능을 제공)

Ex. 맵리듀스 구현중에 입력 데이터를 표현할 수 있는 공통 클래스가 필요하다고 하면
> 맵리듀스 ?
- 구글에서 정보 검색을 위한 데이터 가공을 목적으로 개발된 <br/> 분산 환경에서의 병렬 데이터 처리 기법이자 프로그래밍 모델이다.  

In [6]:
class InputData:
    def read(self):
        raise NotImplementedError
        
class PathInputData(InputData): # 얼마든지 하위 클래스를 만들수 있다.
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

- 각 하위 클래스에서 다시 정의해야만 하는 read 메서드가 들어 있는 공통 클래스를 구현해야 한다.
<br/> Ex. 이 입력 데이터를 소비하는 공통 방법을 제공하는 맵 리듀스 작업자로 쓸 수 있는 추상 인터페이스를 정의하고 싶다.

In [5]:
class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError


class LineCountWorker(Worker): # Work의 구체적인 하위 클래스다.
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')

    def reduce(self, other):
        self.result += other.result

- 잘 작동할 것 처럼 보이지만
<br/> Q. 각 부분을 어떻게 연결해야 할까라는 의문이 생긴다.
<br/> → 인터페이스와 추상화를 제공하는 멋진 클래스를 만들었지만 객체를 생성해 활용해야만 모든 클래스가 쓸모 있다.
<br/> A. 도우미 함수를 활용해서 객체를 만들고 연결하자.

In [ ]:
import os

def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))

- 방금 generate_inputs를 통해 만드는 InputData 인스턴스들을 사용하는 LineCountWorker 인스턴스를 만든다.

In [ ]:
def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers

- 이 Worker 인스턴스의 map 단계를 여러 스레드에 공급해서 실행할 수 있다.
<br/> 그 후 reduce를 반복적으로 호출해서 결과를 최종 값으로 합칠 수 있다.

In [2]:
from threading import Thread


def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads: thread.start()
    for thread in threads: thread.join()

    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

def mapreduce(data_dir): # 함수가 전혀 제너릭 하지 않다는 것이다.
    inputs = generate_inputs(data_dir)
    workers = create_workers(inputs)
    return execute(workers)

import os
import random

def write_test_files(tmpdir):
    os.makedirs(tmpdir)
    for i in range(100):
        with open(os.path.join(tmpdir, str(i)), 'w') as f:
            f.write('\n' * random.randint(0, 100))

tmpdir = 'test_inputs'
write_test_files(tmpdir)

result = mapreduce(tmpdir)
print(f'총 {result} 줄이 있습니다.')


총 5333 줄이 있습니다.


<hr/>

- 문제가 있다. 함수가 전혀 제너릭을 사용하지 않는다.

>제너릭 ?
- 객체 내부에서 사용할 데이터 타입을 외부에서 정하는 기법이다.
- 제네릭은 객체의 타입을 고정하지 않고, 다양한 타입의 객체를 가지게 해준다.

- 다른 InputData나 Worker 하위 클래스를 사용하고 싶다면 
<br/> 각 하위 클래스에 맞게 generate_inputs, create_workers, mapreduce를 재작성해야 한다.

<br/> A. 가장 좋은 방법은 클래스 메서드 다형성을 사용하는 것이다.
- InputData.read에서 사용했던 인스턴스 메서드의 다형성과 똑같은데
<br/> 클래스로 만들어낸 개별 객체에 적용되지 않고 클래스 전체에 적용된다는 점만 다르다.
- 특히 cls를 사용하면 메서드 안에서 현재 클래스의 인스턴스를 만들 수도 있다.

In [ ]:
class GenericInputData:
    def read(self):
        raise NotImplementedError

    @classmethod
    def generate_inputs(cls, config): # 입력 파일 디렉토리 찾기
        raise NotImplementedError

In [8]:
class PathInputData(GenericInputData): # cls()는 PathInputData
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        with open(self.path) as f:
            return f.read()

    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

In [ ]:
class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self, other):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers

- input_class.generate_inputs 호출이 바로 여기서 보여주려는 클래스 다형성의 예이다.
<br/> → create_workers가 __ init__ 메서드를 직접 호출하지 않고 cls()를 호출함으로써
<br/> 다른 방법으로 GenericWorker 객체를 만들 수 있다는 것도 알 수 있다.

In [3]:
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')

    def reduce(self, other):
        self.result += other.result

def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

config = {'data_dir': tmpdir}
result = mapreduce(LineCountWorker, PathInputData, config)
print(f'총 {result} 줄이 있습니다.')

총 5333 줄이 있습니다.


- 구제적인 GenericWorker 하위 클래스에 미치는 영향은 부모 클래스를 바꾸는 것
<br/> mapreducer 함수가  create_workers를 호출하게 변경해서 완전한 제너릭 함수로 만들 수 있다.
<br/><br/> 앞에 있었던 예제와 똑같은 결과지만 제너릭하게 작동해야하므로
<br/> mapreduce 함수에 더 많은 파라미터를 넘겨야 한다.
<br/> → 결합하는 코드를 변경하지 않아도 GenericInputData와 GenericWorker의 하위 클래스를 내가 원한대로 작성

### 기억하자
- 파이썬의 클래스에는 생성자가 __ init__ 메서드 뿐이다.
- @classmethod를 사용하면 클래스에 다른 생성자를 정의할 수 있다.
- 다형성을 활용하면 여러 구체적인 하위 클래스의 객체를 만들고 연결하는 제너릭한 방법을 제공할 수 있다.